In [17]:
import sys

shared_path = r"C:\Users\windows\projects\hackathon\shared"
sys.path.append(shared_path)


In [18]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from gemini_processor import  GeminiOptions, GeminiProcessor

# 環境変数の読み込み
load_dotenv()

# APIキーの確認
if not os.getenv('GOOGLE_API_KEY'):
    raise ValueError('GOOGLE_API_KEYが設定されていません。.envファイルを確認してください。')


In [ ]:
def save_to_bigquery(df, project_id, dataset_id, table_id, if_exists='append'):
    """
    DataFrameをBigQueryに保存する関数
    
    Parameters:
    -----------
    df : pandas.DataFrame
        保存するデータフレーム
    project_id : str
        BigQueryのプロジェクトID
    dataset_id : str
        データセットID
    table_id : str
        テーブルID
    if_exists : str, default 'append'
        テーブルが既に存在する場合の動作
        - 'fail': エラーを発生させる
        - 'replace': テーブルを置き換える
        - 'append': データを追加する
    
    Returns:
    --------
    None
    """
    try:
        # テーブル名の完全なパスを作成
        table_path = f"{project_id}.{dataset_id}.{table_id}"
        
        # DataFrameをBigQueryに保存
        df.to_gbq(
            destination_table=table_path,
            project_id=project_id,
            if_exists=if_exists,
            location='asia-northeast1'  # 東京リージョン
        )
        print(f"データを {table_path} に保存しました。")
        
    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")
        raise

In [19]:
response_schema = {
    "$schema": "https://json-schema.org/draft/2020-12/schema",
    "title": "MealPlan",
    "type": "object",
    "properties": {
        "title": { "type": "string" },
        "cuisine": { "type": "string" },
        "total_time_min": {
            "type": "integer",
            "minimum": 0
        },
        "nutrition": { "$ref": "#/$defs/nutrition" },
        "ingredients": {
            "type": "array",
            "items": { "$ref": "#/$defs/ingredient" }
        },
        "instructions": {
            "type": "array",
            "items": { "type": "string" },
            "minItems": 1
        },
        "notes": { "type": "string" }
    },
    "required": [
        "title",
        "cuisine",
        "total_time_min",
        "nutrition",
        "ingredients",
        "instructions"
    ],
    "additionalProperties": False,

    "$defs": {
        "nutrition": {
            "type": "object",
            "properties": {
                "kcal":       { "type": "number", "minimum": 0 },
                "protein_g":  { "type": "number", "minimum": 0 },
                "fat_g":      { "type": "number", "minimum": 0 },
                "carb_g":     { "type": "number", "minimum": 0 },
                "salt_g":     { "type": "number", "minimum": 0 }
            },
            "required": ["kcal", "protein_g", "fat_g", "carb_g", "salt_g"],
            "additionalProperties": False
        },

        "ingredient": {
            "type": "object",
            "properties": {
                "name":     { "type": "string" },
                "quantity": { "type": "string" },
                "unit":     { "type": "string" }
            },
            "required": ["name", "quantity", "unit"],
            "additionalProperties": False
        }
    }
}

In [20]:
# 基本的な使用方法
schema_tool = {"type": "response_schema", "schema": response_schema}

opts = GeminiOptions(
    temperature=0.3,
    max_output_tokens=10000,
    tools=None                             
)

processor = GeminiProcessor(options=opts)

In [21]:
def extract_meals(result_df, output_col="output_parsed"):
    """
    result_df の output_col に格納された JSON 文字列（または dict）から
    meals 情報を行ベースで取り出して DataFrame を返す。

    返却列
    -------
    row_id             : 元の result_df の行インデックス
    period             : 献立対象期間
    date               : 各食事の日付
    meal_type          : 朝食 / 昼食 / 夕食
    title              : メニュー名
    cuisine            : 和食 / 洋食 ...
    total_time_min     : 調理時間
    kcal, protein_g,
    fat_g, carb_g,
    salt_g             : 栄養値
    """
    meal_rows = []

    for idx, raw in result_df[output_col].dropna().items():
        # --- ① JSON を dict 化 -------------------------------------------------
        if isinstance(raw, dict):
            data = raw
        else:
            txt = str(raw).strip()
            # 先頭末尾に ```json ... ``` が付いているケースを除去
            if txt.startswith("```"):
                txt = txt.lstrip("`").strip()
                if txt.lower().startswith("json"):
                    txt = txt[4:].lstrip()  # "json" の後の改行を削除
            data = json.loads(txt)

        # --- ② meals 部分をフラット化 -----------------------------------------
        for meal in data.get("meals", []):
            meal_flat = {
                "row_id": idx,
                "period": data.get("period"),
                "date": meal["date"],
                "meal_type": meal["meal_type"],
                "title": meal["title"],
                "cuisine": meal["cuisine"],
                "total_time_min": meal["total_time_min"],
                **meal["nutrition"],              # kcal などを展開
            }
            meal_rows.append(meal_flat)

    return pd.DataFrame(meal_rows)

In [22]:
system_prompt = f"""
あなたは「パーソナル栄養プランナーAI」です。
以下の条件に従って、1食分の献立をJSON形式で出力してください。

【役割】
- 目的: ユーザーの食事目標を最小の手間で達成する献立を作成する。

【必須条件】
1. 栄養基準を満たす（後述のnutrition_targets）
2. 禁止食材／アレルゲンを含まない（後述のfood_restrictions）
3. 在庫優先: 冷蔵庫在庫（後述のinventory）を優先的に消費
4. 調理時間と調理器具制約（後述のtime_limit, equipment）を守る
5. 指定ジャンル（後述のcuisine_genres）を守る

【出力形式】
必ず以下のJSON形式で出力してください（コードブロックは不要）：
{{
    "title": "メニュー名",
    "cuisine": "和食",
    "total_time_min": 25,  // 整数値で分単位
    "nutrition": {{
        "kcal": 560,       // 数値（kcal）
        "protein_g": 30,   // 数値（g）
        "fat_g": 20,       // 数値（g）
        "carb_g": 45,      // 数値（g）
        "salt_g": 2.5      // 数値（g）
    }},
    "instructions": [
        "手順1",
        "手順2"
    ],
    "ingredients": [
        {{
            "name": "鶏胸肉",
            "quantity": "200",
            "unit": "g"
        }},
        {{
            "name": "塩麹",
            "quantity": "大さじ1.5",
            "unit": ""
        }},
        {{
            "name": "ほうれん草",
            "quantity": "100",
            "unit": "g"
        }}
    ]
}}

注意事項：
- nutritionの値は必ず数値型で出力（単位は含めない）
- ingredientsは必ずオブジェクトの配列として出力
- total_time_minは必ず整数値で出力
"""

In [23]:
#テーブルの指定
project_name = 'dal-ai-agent-hackathon'
dataset_name = 'meal_planner'

table_name = 'demo'
demo_table = f'{project_name}.{dataset_name}.{table_name}'

table_name = 'fridge_items'
fridge_items_table = f'{project_name}.{dataset_name}.{table_name}'

table_name = 'recipe_ingredients'
recipe_ingredients_table = f'{project_name}.{dataset_name}.{table_name}'


# SQL クエリを一つにまとめる
demo_query = f"""
SELECT *
FROM `{demo_table}`
"""

fridge_items_query = f"""
SELECT *
FROM `{fridge_items_table}`
"""

recipe_ingredients_query = f"""
SELECT *
FROM `{recipe_ingredients_table}`
"""



In [24]:
demo = pd.read_gbq(demo_query.replace('\n', ' ').replace('\u3000', ''), project_name, dialect='standard')
fridge_items = pd.read_gbq(fridge_items_query.replace('\n', ' ').replace('\u3000', ''), project_name, dialect='standard')
recipe_ingredients = pd.read_gbq(recipe_ingredients_query.replace('\n', ' ').replace('\u3000', ''), project_name, dialect='standard')

C:\Users\windows\AppData\Local\Temp\ipykernel_10984\1755557862.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  demo = pd.read_gbq(demo_query.replace('\n', ' ').replace('\u3000', ''), project_name, dialect='standard')
C:\Users\windows\AppData\Local\Temp\ipykernel_10984\1755557862.py:2: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  fridge_items = pd.read_gbq(fridge_items_query.replace('\n', ' ').replace('\u3000', ''), project_name, dialect='standard')
C:\Users\windows\AppData\Local\Temp\ipykernel_10984\1755557862.py:3: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#panda

In [25]:
demo

,user_id,name,age,gender,dietary_style,created_at
0,user_003,鈴木 美咲,22,female,vegan,2025-05-25 04:53:45.207177+00:00
1,user_007,伊藤 結衣,30,female,vegetarian,2025-05-25 04:53:45.207177+00:00
2,user_009,渡辺 花音,27,female,low_fat,2025-05-25 04:53:45.207177+00:00
3,user_005,山田 杏子,31,female,gluten_free,2025-05-25 04:53:45.207177+00:00
4,user_001,上原 おもち,28,female,balanced,2025-05-25 04:53:45.207177+00:00
5,user_006,高橋 慎吾,26,male,high_protein,2025-05-25 04:53:45.207177+00:00
6,user_010,小林 亮介,33,male,balanced,2025-05-25 04:53:45.207177+00:00
7,user_008,中村 拓海,38,male,balanced,2025-05-25 04:53:45.207177+00:00
8,user_004,田中 直樹,41,male,balanced,2025-05-25 04:53:45.207177+00:00
9,user_002,佐藤 太郎,35,male,low_carb,2025-05-25 04:53:45.207177+00:00


In [26]:

# 要件に合わせてあとで変更

human_prompt = f'''
nutrition_targets:
  kcal_per_day: 2100
  protein_g: 110
  fat_g: 55
  carb_g: 240
  salt_g: 6
food_restrictions:
  allergens: ["えび", "かに", "落花生"]
  dislikes: ["パクチー"]
cuisine_genres:
  preferences: "和食"
time_limit:
  breakfast_max_min: 15
  lunch_max_min: 20
  dinner_max_min: 40
equipment: ["電子レンジ", "IHコンロ2口", "オーブン無し"]
inventory:
  - name: "鶏胸肉", quantity_g: 400
  - name: "ほうれん草", quantity_g: 100
  - name: "卵", quantity: 6
budget_jpy: 5000
servings: 1
special_notes: "減量中なので夕食の糖質は控えめに"
'''

In [27]:
data = {
    "human_prompt": [human_prompt],  # リストに変換
    "system_prompt": [system_prompt]  # リストに変換
}
df = pd.DataFrame(data)

In [28]:
# Gemini API 呼び出し
result_df = processor.process_dataframe(df, human_prompt_col="human_prompt", system_prompt_col="system_prompt", output_col="output")
result_df

,human_prompt,system_prompt,output
0,\nnutrition_targets:\n kcal_per_day: 2100\n ...,\nあなたは「パーソナル栄養プランナーAI」です。\n以下の条件に従って、1食分の献立をJS...,"{\n ""title"": ""鶏むね肉の低糖質照り焼き定食"",\n ""cuisin..."


In [29]:
print(result_df["output"][0])

{
    "title": "鶏むね肉の低糖質照り焼き定食",
    "cuisine": "和食",
    "total_time_min": 30,
    "nutrition": {
        "kcal": 510,
        "protein_g": 50,
        "fat_g": 8,
        "carb_g": 46,
        "salt_g": 2.1
    },
    "instructions": [
        "鶏むね肉は厚さ1.5cm程度のそぎ切りにする。生姜はすりおろす。",
        "フライパンにサラダ油を熱し、鶏むね肉を並べ、中火で両面をこんがりと焼く。",
        "鶏肉に火が通ったら、醤油（小さじ2）、料理酒（大さじ1）、みりん（小さじ1）、水（大さじ1）、すりおろし生姜を混ぜたタレを加えて煮絡める。",
        "ほうれん草は根元を切り落とし、沸騰した湯でさっと茹でて冷水にとり、水気をしっかり絞って3cm長さに切る。",
        "ボウルにだし汁（大さじ2）と醤油（小さじ1）を混ぜ、ほうれん草を和えておひたしにする。",
        "耐熱容器に卵を割り入れ、黄身に数カ所穴を開け、水を小さじ1程度加えてラップをせずに電子レンジ（600W）で30秒～1分加熱し、温泉卵を作る。（加熱時間は調整）",
        "ご飯を茶碗に盛り、鶏むね肉の照り焼き、ほうれん草のおひたし、温泉卵を添えて完成。"
    ],
    "ingredients": [
        {
            "name": "鶏むね肉",
            "quantity": "180",
            "unit": "g"
        },
        {
            "name": "ほうれん草",
            "quantity": "100",
            "unit": "g"
        },
        {
            "name": "卵",
            "quantity": "1",
            "unit": "個"
   

In [30]:
def parse_menu_json(raw_json: str | dict):
    """
    鶏胸肉… のようなメニュー JSON を
    ├ header_df      : タイトル／ジャンル／調理時間
    ├ nutrition_df   : 栄養値（1行1列ずつ）
    ├ ingredients_df : 材料（name, quantity, unit）
    └ instructions_df: 手順   （step, text）
    という4つの DataFrame に分解して返す
    """
    data = raw_json if isinstance(raw_json, dict) else json.loads(raw_json)

    header_df = pd.DataFrame([{
        "title":           data["title"],
        "cuisine":         data["cuisine"],
        "total_time_min":  data["total_time_min"],
    }])

    # "カロリー: 560kcal" → {"カロリー": "560kcal", ...}
    nutr = dict(s.split(":", 1) for s in data["nutrition"])
    nutrition_df = pd.DataFrame([nutr])

    ingredients_df = pd.DataFrame(
        data["ingredients"],
        columns=["name", "quantity", "unit"]
    )

    instructions_df = pd.DataFrame(
        {"step": range(1, len(data["instructions"])+1),
         "text": data["instructions"]}
    )

    return header_df, nutrition_df, ingredients_df, instructions_df

In [31]:
def parse_menu_json(raw_json: str | dict):
    """
    メニューJSONを
    ├ header_df      : タイトル／ジャンル／調理時間
    ├ nutrition_df   : 栄養値
    ├ ingredients_df : 材料（name, quantity, unit）
    └ instructions_df: 手順（step, text）
    という4つのDataFrameに分解して返す
    """
    data = raw_json if isinstance(raw_json, dict) else json.loads(raw_json)

    # ヘッダー情報
    header_df = pd.DataFrame([{
        "title":           data["title"],
        "cuisine":         data["cuisine"],
        "total_time_min":  data["total_time_min"],
    }])

    # 栄養情報（すでにオブジェクト形式）
    nutrition_df = pd.DataFrame([data["nutrition"]])

    # 材料情報
    ingredients_df = pd.DataFrame(data["ingredients"])

    # 手順
    instructions_df = pd.DataFrame({
        "step": range(1, len(data["instructions"])+1),
        "text": data["instructions"]
    })

    return header_df, nutrition_df, ingredients_df, instructions_df

In [32]:
header, nutr, ingred, instr = parse_menu_json(result_df.loc[0, "output"])


In [39]:
save_to_bigquery(
    df=header,
    project_id=project_name,
    dataset_id=dataset_name,
    table_id='header',
    if_exists='append'
)

C:\Users\windows\AppData\Local\Temp\ipykernel_10984\3689274653.py:30: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 15827.56it/s]

データを dal-ai-agent-hackathon.meal_planner.header に保存しました。


In [41]:
save_to_bigquery(
    df=nutr,
    project_id=project_name,
    dataset_id=dataset_name,
    table_id='nutr',
    if_exists='append'
)

C:\Users\windows\AppData\Local\Temp\ipykernel_10984\3689274653.py:30: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 25575.02it/s]

データを dal-ai-agent-hackathon.meal_planner.nutr に保存しました。


In [42]:
save_to_bigquery(
    df=ingred,
    project_id=project_name,
    dataset_id=dataset_name,
    table_id='ingred',
    if_exists='append'
)

C:\Users\windows\AppData\Local\Temp\ipykernel_10984\3689274653.py:30: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 31300.78it/s]

データを dal-ai-agent-hackathon.meal_planner.ingred に保存しました。


In [43]:
save_to_bigquery(
    df=instr,
    project_id=project_name,
    dataset_id=dataset_name,
    table_id='instr',
    if_exists='append'
)

C:\Users\windows\AppData\Local\Temp\ipykernel_10984\3689274653.py:30: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 12595.51it/s]

データを dal-ai-agent-hackathon.meal_planner.instr に保存しました。
